In [1]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


## Set up parameters, create isochrone grid

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.MISTv1()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F182M']
filters = ['m_jwst_F115W', 'm_jwst_F182M']
iso_dir = 'isochrones/'
metallicity = 0

In [3]:
# read nbody data
path_to_data = '../../../../../../storage/jfarias/Data/Orion/massive_clusters/M3000new/sigma0p1/fiducial/sfe_ff001/00/'
snapshot = Reader.read_snapshot(path_to_data, time = 20)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()

# read ages for simulated stars
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass        isMultiple        compMass             systemMass              e                 log_a               age        
------------------- ---------- ---------------------- ------------------- -------------------- ------------------ ------------------
0.06895842179945424        0.0                     [] 0.06895842179945424                  nan                nan  18.62630555946738
0.08626887187757219        0.0                     [] 0.08626887187757219                  nan                nan  3.069082513027311
  2.326448478984193        0.0                     []   2.326448478984193                  nan                nan  18.33038253724003
0.04093112166717806        0.0                     [] 0.04093112166717806                  nan                nan  3.069082513027311
                ...        ...                    ...                 ...                  ...                ...                ...
0.03156546631062232        1.0 [0.017415375068988628] 0.0489808413796

In [4]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 6.407048843194058
Max log age = 7.3018036073193215
Min level age = 2552988.4099163027
Max level age = 20035657.88269043


In [5]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
print(compMasses)

1745
[0.018772196398952473, 0.020398101994065967, 0.04970394506596586, 0.011418578712969283, 0.1109845537581072, 0.08288741359987532, 0.054993759410529613, 0.33087717935666183, 0.17271523389867505, 0.013492928385704234, 0.06711686707284298, 0.10798400551211244, 0.029492553417922056, 0.14367710080576154, 0.26903433005844346, 0.08474773851417283, 0.30095260930078693, 0.05497012322155322, 0.09173333630199015, 0.025891768717209534, 0.02735674785778421, 0.1558841429805367, 0.048604116260690874, 0.04128694062892624, 0.03487493039755069, 0.14196306235013237, 0.04280274224495466, 0.02310867140421406, 0.3955789882313527, 0.15676043958471955, 0.08859345991063905, 0.055069286248966655, 0.1552959762503858, 0.09515570587436127, 0.04718531929120123, 0.03205825521948448, 0.011501610127550384, 1.1694443225860596, 0.0922981857375905, 0.11274085938930511, 0.16108334558162823, 0.07456852106262159, 0.013275199569761753, 0.11918815425644891, 0.038134801276114384, 0.019446755405924193, 0.10889488258261569, 

In [6]:
start = (min(level_ages) // 0.5e6) * 0.5e6
end = (max(level_ages) // 0.5e6) * 0.5e6 + 2 * 0.5e6
increment = 0.5e6  # 500,000

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[6.39794001 6.47712125 6.54406804 6.60205999 6.65321251 6.69897
 6.74036269 6.77815125 6.81291336 6.84509804 6.87506126 6.90308999
 6.92941893 6.95424251 6.97772361 7.         7.0211893  7.04139269
 7.06069784 7.07918125 7.09691001 7.11394335 7.13033377 7.14612804
 7.161368   7.17609126 7.1903317  7.20411998 7.21748394 7.23044892
 7.24303805 7.25527251 7.26717173 7.2787536  7.29003461 7.30103
 7.31175386]
[ 2500000.  3000000.  3500000.  4000000.  4500000.  5000000.  5500000.
  6000000.  6500000.  7000000.  7500000.  8000000.  8500000.  9000000.
  9500000. 10000000. 10500000. 11000000. 11500000. 12000000. 12500000.
 13000000. 13500000. 14000000. 14500000. 15000000. 15500000. 16000000.
 16500000. 17000000. 17500000. 18000000. 18500000. 19000000. 19500000.
 20000000. 20500000.]


In [ ]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

Changing to logg=4.50 for T= 39293 logg=4.00
Changing to logg=4.50 for T= 39737 logg=3.99
Changing to logg=4.50 for T= 40206 logg=3.98
Changing to logg=4.50 for T= 40487 logg=3.97
Changing to logg=4.50 for T= 40764 logg=3.97
Changing to logg=4.50 for T= 41036 logg=3.95
Changing to logg=4.50 for T= 41298 logg=3.94
Changing to logg=4.50 for T= 41548 logg=3.93
Changing to logg=4.50 for T= 41810 logg=3.92
Changing to logg=4.50 for T= 42053 logg=3.90
Changing to logg=4.50 for T= 42247 logg=3.88
Changing to logg=4.50 for T= 42389 logg=3.86
Changing to logg=4.50 for T= 42598 logg=3.83
Changing to logg=4.50 for T= 43046 logg=3.82
Changing to logg=4.50 for T= 44070 logg=3.81
Changing to logg=4.50 for T= 44382 logg=3.80
Changing to logg=4.50 for T= 44435 logg=3.80
Changing to logg=4.50 for T= 44490 logg=3.79
Changing to logg=4.50 for T= 44532 logg=3.78
Changing to T= 50000 for T= 55470 logg=4.05
Changing to logg=5.00 for T= 55470 logg=4.05
Changing to T= 50000 for T= 56402 logg=4.07
Changing to 

Changing to T= 50000 for T= 67022 logg=4.28
Changing to logg=5.00 for T= 67022 logg=4.28
Changing to T= 50000 for T= 67541 logg=4.29
Changing to logg=5.00 for T= 67541 logg=4.29
Changing to T= 50000 for T= 68089 logg=4.30
Changing to logg=5.00 for T= 68089 logg=4.30
Changing to T= 50000 for T= 68666 logg=4.32
Changing to logg=5.00 for T= 68666 logg=4.32
Changing to T= 50000 for T= 69275 logg=4.33
Changing to logg=5.00 for T= 69275 logg=4.33
Changing to T= 50000 for T= 69901 logg=4.35
Changing to logg=5.00 for T= 69901 logg=4.35
Changing to T= 50000 for T= 70543 logg=4.36
Changing to logg=5.00 for T= 70543 logg=4.36
Changing to T= 50000 for T= 71202 logg=4.38
Changing to logg=5.00 for T= 71202 logg=4.38
Changing to T= 50000 for T= 71872 logg=4.39
Changing to logg=5.00 for T= 71872 logg=4.39
Changing to T= 50000 for T= 72550 logg=4.41
Changing to logg=5.00 for T= 72550 logg=4.41
Changing to T= 50000 for T= 73235 logg=4.42
Changing to logg=5.00 for T= 73235 logg=4.42
Changing to T= 50000 

Changing to T= 50000 for T=112623 logg=5.09
Changing to logg=5.00 for T=112623 logg=5.09
Changing to T= 50000 for T=112616 logg=5.09
Changing to logg=5.00 for T=112616 logg=5.09
Changing to T= 50000 for T=112607 logg=5.09
Changing to logg=5.00 for T=112607 logg=5.09
Changing to T= 50000 for T=112597 logg=5.09
Changing to logg=5.00 for T=112597 logg=5.09
Changing to T= 50000 for T=112587 logg=5.09
Changing to logg=5.00 for T=112587 logg=5.09
Changing to T= 50000 for T=112576 logg=5.09
Changing to logg=5.00 for T=112576 logg=5.09
Changing to T= 50000 for T=112564 logg=5.09
Changing to logg=5.00 for T=112564 logg=5.09
Changing to T= 50000 for T=112551 logg=5.09
Changing to logg=5.00 for T=112551 logg=5.09
Changing to T= 50000 for T=112538 logg=5.09
Changing to logg=5.00 for T=112538 logg=5.09
Changing to T= 50000 for T=112524 logg=5.09
Changing to logg=5.00 for T=112524 logg=5.09
Changing to T= 50000 for T=112509 logg=5.09
Changing to logg=5.00 for T=112509 logg=5.09
Changing to T= 50000 

Changing to T= 50000 for T=110128 logg=5.02
Changing to logg=5.00 for T=110128 logg=5.02
Changing to T= 50000 for T=110096 logg=5.02
Changing to logg=5.00 for T=110096 logg=5.02
Changing to T= 50000 for T=110065 logg=5.02
Changing to logg=5.00 for T=110065 logg=5.02
Changing to T= 50000 for T=110034 logg=5.02
Changing to logg=5.00 for T=110034 logg=5.02
Changing to T= 50000 for T=110003 logg=5.02
Changing to logg=5.00 for T=110003 logg=5.02
Changing to T= 50000 for T=109972 logg=5.02
Changing to logg=5.00 for T=109972 logg=5.02
Changing to T= 50000 for T=109942 logg=5.02
Changing to logg=5.00 for T=109942 logg=5.02
Changing to T= 50000 for T=109912 logg=5.01
Changing to logg=5.00 for T=109912 logg=5.01
Changing to T= 50000 for T=109882 logg=5.01
Changing to logg=5.00 for T=109882 logg=5.01
Changing to T= 50000 for T=109852 logg=5.01
Changing to logg=5.00 for T=109852 logg=5.01
Changing to T= 50000 for T=109823 logg=5.01
Changing to logg=5.00 for T=109823 logg=5.01
Changing to T= 50000 

M = 103.009 Msun  T = 58412 K  m_jwst_F182M = 3.59
M = 139.953 Msun  T = 112608 K  m_jwst_F182M = 6.65
M = 140.007 Msun  T = 110414 K  m_jwst_F182M = 6.47
      Time taken: 30.40 seconds
Changing to logg=4.00 for T= 35119 logg=4.00
Changing to logg=4.00 for T= 35444 logg=3.99
Changing to logg=4.00 for T= 35762 logg=3.98
Changing to logg=4.00 for T= 36158 logg=3.96
Changing to logg=4.00 for T= 36596 logg=3.94
Changing to logg=4.00 for T= 36831 logg=3.93
Changing to logg=4.00 for T= 37051 logg=3.92
Changing to logg=4.00 for T= 37222 logg=3.90
Changing to logg=4.00 for T= 37366 logg=3.89
Changing to logg=4.00 for T= 37483 logg=3.87
Changing to logg=4.00 for T= 37567 logg=3.85
Changing to logg=4.00 for T= 37806 logg=3.83
Changing to logg=4.00 for T= 37901 logg=3.81
Changing to logg=4.00 for T= 37907 logg=3.79
Changing to logg=4.00 for T= 37889 logg=3.77
Changing to logg=4.00 for T= 37847 logg=3.76
Changing to logg=4.00 for T= 37778 logg=3.74
Changing to logg=4.00 for T= 37685 logg=3.72
Cha

...

Changing to T= 50000 for T=177701 logg=5.79
Changing to logg=5.00 for T=177701 logg=5.79
Changing to T= 50000 for T=178673 logg=5.79
Changing to logg=5.00 for T=178673 logg=5.79
Changing to T= 50000 for T=179649 logg=5.80
Changing to logg=5.00 for T=179649 logg=5.80
Changing to T= 50000 for T=180629 logg=5.81
Changing to logg=5.00 for T=180629 logg=5.81
Changing to T= 50000 for T=181614 logg=5.82
Changing to logg=5.00 for T=181614 logg=5.82
Changing to T= 50000 for T=182604 logg=5.83
Changing to logg=5.00 for T=182604 logg=5.83
Changing to T= 50000 for T=183597 logg=5.84
Changing to logg=5.00 for T=183597 logg=5.84
Changing to T= 50000 for T=184594 logg=5.84
Changing to logg=5.00 for T=184594 logg=5.84
Changing to T= 50000 for T=185595 logg=5.85
Changing to logg=5.00 for T=185595 logg=5.85
Changing to T= 50000 for T=186600 logg=5.86
Changing to logg=5.00 for T=186600 logg=5.86
Changing to T= 50000 for T=187608 logg=5.87
Changing to logg=5.00 for T=187608 logg=5.87
Changing to T= 50000 

M =   2.409 Msun  T = 10007 K  m_jwst_F115W = 8.88
M =   7.159 Msun  T = 20933 K  m_jwst_F115W = 7.16
M =  49.655 Msun  T = 29749 K  m_jwst_F115W = 1.85
M =  56.931 Msun  T = 16123 K  m_jwst_F115W = -0.32
M =  56.978 Msun  T =  8296 K  m_jwst_F115W = -2.24
M =  64.399 Msun  T = 68333 K  m_jwst_F115W = 3.96
M =  69.710 Msun  T = 223651 K  m_jwst_F115W = 9.18
Starting filter: jwst,F182M   Elapsed time: 21.26 seconds
Starting synthetic photometry
M =   0.105 Msun  T =  2998 K  m_jwst_F182M = 14.19
M =   2.409 Msun  T = 10007 K  m_jwst_F182M = 8.90
M =   7.159 Msun  T = 20933 K  m_jwst_F182M = 7.31
M =  49.655 Msun  T = 29749 K  m_jwst_F182M = 2.05
M =  56.931 Msun  T = 16123 K  m_jwst_F182M = -0.22
M =  56.978 Msun  T =  8296 K  m_jwst_F182M = -2.29
M =  64.399 Msun  T = 68333 K  m_jwst_F182M = 4.18
M =  69.710 Msun  T = 223651 K  m_jwst_F182M = 9.40
      Time taken: 38.06 seconds
Changing to logg=4.00 for T= 32444 logg=3.99
Changing to logg=4.00 for T= 32684 logg=3.98
Changing to logg=4

Changing to logg=2.50 for T= 12800 logg=1.61
Changing to logg=2.50 for T= 12691 logg=1.60
Changing to logg=2.50 for T= 12584 logg=1.58
Changing to logg=2.50 for T= 12477 logg=1.57
Changing to logg=2.50 for T= 12371 logg=1.55
Changing to logg=2.50 for T= 12266 logg=1.54
Changing to logg=2.50 for T= 12162 logg=1.52
Changing to logg=2.50 for T= 12059 logg=1.51
Changing to logg=2.50 for T= 11956 logg=1.49
Changing to logg=2.50 for T= 11855 logg=1.48
Changing to logg=2.50 for T= 11754 logg=1.46
Changing to logg=2.00 for T= 11654 logg=1.45
Changing to logg=2.00 for T= 11556 logg=1.44
Changing to logg=2.00 for T= 11457 logg=1.42
Changing to logg=2.00 for T= 11360 logg=1.41
Changing to logg=2.00 for T= 11264 logg=1.39
Changing to logg=2.00 for T= 11168 logg=1.38
Changing to logg=2.00 for T= 11073 logg=1.36
Changing to logg=2.00 for T= 10979 logg=1.35
Changing to logg=2.00 for T= 10886 logg=1.33
Changing to logg=2.00 for T= 10794 logg=1.32
Changing to logg=2.00 for T= 10702 logg=1.30
Changing t

Changing to T= 50000 for T=107260 logg=4.96
Changing to logg=5.00 for T=107260 logg=4.96
Changing to T= 50000 for T=107728 logg=4.97
Changing to logg=5.00 for T=107728 logg=4.97
Changing to T= 50000 for T=108209 logg=4.97
Changing to logg=5.00 for T=108209 logg=4.97
Changing to T= 50000 for T=108703 logg=4.98
Changing to logg=5.00 for T=108703 logg=4.98
Changing to T= 50000 for T=109198 logg=4.99
Changing to logg=5.00 for T=109198 logg=4.99
Changing to T= 50000 for T=109693 logg=4.99
Changing to logg=5.00 for T=109693 logg=4.99
Changing to T= 50000 for T=110185 logg=5.00
Changing to logg=5.00 for T=110185 logg=5.00
Changing to T= 50000 for T=110678 logg=5.00
Changing to logg=5.00 for T=110678 logg=5.00
Changing to T= 50000 for T=111171 logg=5.01
Changing to logg=5.00 for T=111171 logg=5.01
Changing to T= 50000 for T=111665 logg=5.02
Changing to logg=5.00 for T=111665 logg=5.02
Changing to T= 50000 for T=112160 logg=5.02
Changing to logg=5.00 for T=112160 logg=5.02
Changing to T= 50000 

Isochrone generation took 61.531726 s.
Making photometry for isochrone: log(t) = 6.65  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:09:33.121091   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.107 Msun  T =  3003 K  m_jwst_F115W = 15.04
M =   2.306 Msun  T =  9312 K  m_jwst_F115W = 9.04
M =   7.092 Msun  T = 20808 K  m_jwst_F115W = 7.17
M =  42.523 Msun  T = 28376 K  m_jwst_F115W = 2.01
M =  46.065 Msun  T = 15044 K  m_jwst_F115W = -0.15
M =  46.118 Msun  T =  6418 K  m_jwst_F115W = -2.48
M =  50.747 Msun  T = 38771 K  m_jwst_F115W = 2.24
M =  53.670 Msun  T = 130667 K  m_jwst_F115W = 7.10
Starting filter: jwst,F182M   Elapsed time: 18.07 seconds
Starting synthetic photometry
M =   0.107 Msun  T =  3003 K  m_jwst_F182M = 14.27
M =   2.306 Msun  T =  9312 K  m_jwst_F182M = 9.03
M =   7.092 Msun  T = 20808 K  m_jwst_F182M = 7.32
M =  42.523 Msun  T = 28376 K  m_jwst_F182M = 2.20
M =  46.065 Msun  T = 150

In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/12-18')
import interpolatorExp as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
for i in range(len(stars)):
    print(stars[i])

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

# plot CMD
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

for i in range(len(stars)):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')
